In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,2"

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
tnts_san = SAN(TS=True, TN=True, depth=2)
tnts_san.compile(optimizer='adam', loss='mae')
tnts_san(np.random.random((1, 160, 36, 36, 3)));
tnts_san.summary()

Model: "san"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Sample 0 (Sequential)           │ (160, 18, 18, 64)      │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Sample 1 (Sequential)           │ (160, 9, 9, 32)        │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1 0 (Sequential)            │ (160, 36, 36, 32)      │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1 1 (Sequential)            │ (160, 18, 18, 16)      │        11,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 0 (Sequential)            │ (160, 18, 18, 64)      │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 1 (Sequential)            │ (160, 9, 9, 32)        │        13,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv1 0 (Sequential)        │ (160, 36, 36, 16)      │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv1 1 (Sequential)        │ (160, 18, 18, 8)       │         1,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv2 0 (Sequential)        │ (160, 18, 18, 32)      │        27,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv2 1 (Sequential)        │ (160, 9, 9, 16)        │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask1 0 (Sequential)            │ (160, 36, 36, 1)       │            17 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask1 1 (Sequential)            │ (160, 18, 18, 1)       │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask2 0 (Sequential)            │ (160, 18, 18, 1)       │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask2 1 (Sequential)            │ (160, 9, 9, 1)         │            17 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pooling & Downsample            │ (160, 18, 18, 32)      │             0 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten & Dense (Sequential)    │ (160, 1)               │        65,793 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 209,277 (817.49 KB)

 Trainable params: 209,277 (817.49 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
batch_size = 128

tape = "/root/ssd_cache/rppg_training_data/mmpd_160x36x36_all"

train_tape = load_datatape(tape, fold='train', batch=batch_size)
valid_tape = load_datatape(tape, fold='val', extended_hr='False', batch=batch_size)

training_set, validation_set = KerasDataset(train_tape), KerasDataset(valid_tape)

#training_set = training_set.apply_fn(load_to_gpu) # pre-load
training_set = training_set.apply_fn(compress_aug)
#training_set = training_set.apply_fn(gnoise_aug)

mmpd_160x36x36_all             datatape has been loaded.     7552 items total. fold=train selected
mmpd_160x36x36_all             datatape has been loaded.     1664 items total. fold=val&extended_hr=False selected


In [4]:
#tnts_san.fit(training_set, validation_data=validation_set, epochs=20, callbacks=[keras.callbacks.ModelCheckpoint('../weights/tnts_san_36.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(tnts_san, training_set, validation_set, epochs=20, check_point_path='../weights/tnts_san_36_mmpd.weights.h5')
tnts_san.load_weights('../weights/tnts_san_36_mmpd.weights.h5')

Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch   1:  Training loss: 1.234	Validation loss:0.9641	


Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.7864	Validation loss:0.8232	


Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.7821	Validation loss:0.8257	


In [5]:
eval_on_dataset(dataset_H5_rlap, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TNTS-SAN-F2W_MMPD_RLAP.h5', scenes=['R1', 'R2', 'R3', 'R4'])
get_metrics('../results/TNTS-SAN-F2W_MMPD_RLAP.h5')

  0%|          | 0/220 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,2.107±0.093,6.928±1.999,2.715±0.12,0.82644
Whole,1.114±0.139,2.337±1.185,1.432±0.172,0.97529


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TNTS-SAN-F2W_MMPD_COHFACE.h5', fps=30)
get_metrics('../results/TNTS-SAN-F2W_MMPD_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,4.119±0.222,10.325±3.11,5.543±0.294,0.65694
Whole,3.037±0.494,7.018±3.913,3.985±0.592,0.80897


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TNTS-SAN-F2W_MMPD_PURE.h5')
get_metrics('../results/TNTS-SAN-F2W_MMPD_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.019±0.16,4.528±2.905,1.686±0.3,0.98072
Whole,0.622±0.288,2.296±2.222,1.054±0.561,0.99508


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TNTS-SAN-F2W_MMPD_UBFC.h5')
get_metrics('../results/TNTS-SAN-F2W_MMPD_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.354±0.095,2.549±1.12,1.39±0.096,0.98876
Whole,0.546±0.093,0.815±0.456,0.553±0.09,0.99898
